In [1]:
import sys
import xlsxwriter
import pandas as pd
import sys
import mysql.connector
import os
from dotenv import load_dotenv
from datetime import date

today = date.today()
load_dotenv()

#configurar la conexion a la base de datos
DB_USERNAME = os.getenv('DB_USERNAME')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')
DB_HOST=os.getenv('DB_HOST')

# Conectar a DB
cnx = mysql.connector.connect(user=DB_USERNAME,
                              password=DB_PASSWORD,
                              host=DB_HOST,
                              port=DB_PORT,
                              database=DB_DATABASE,
                              use_pure=False)


In [4]:
e=pd.read_sql('select * from respuestas20 where completed=1 and aplica!=111',cnx)

In [5]:
import datetime
data1 = e['created_at'].values[0]
data2 = e['updated_at'].values[0]

diff = data2 - data1

days, seconds = diff.days, diff.seconds
hours = days * 24 + seconds // 3600
minutes = (seconds % 3600) // 60
seconds = seconds % 60

print( hours,minutes,seconds)

AttributeError: 'numpy.timedelta64' object has no attribute 'days'

In [8]:
diff

numpy.timedelta64(3305000000000,'ns')

In [9]:
e['created_at'].values[0]

numpy.datetime64('2024-02-02T20:26:57.000000000')

In [56]:
e['completed'].unique()

array([1], dtype=int64)

In [23]:
(pd.Timestamp(e['updated_at'].values[0])-pd.Timestamp(e['created_at'].values[0])).seconds/60

55.083333333333336

In [27]:
e=e.assign(tiempo=0)

In [28]:
for i in range(len(e)):
    e['tiempo'].values[i]=(pd.Timestamp(e['updated_at'].values[i])-pd.Timestamp(e['created_at'].values[i])).seconds/60

In [32]:
e[['aplica','tiempo','fec_capt','created_at','updated_at']]

,aplica,tiempo,fec_capt,created_at,updated_at
0,22,55,2024-02-02,2024-02-02 20:26:57,2024-02-02 21:22:02
1,15,31,2024-02-02,2024-02-02 21:13:31,2024-02-02 21:44:54
2,15,20,2024-02-02,2024-02-02 22:33:19,2024-02-02 22:54:02
3,17,17,2024-02-02,2024-02-06 15:17:57,2024-02-06 15:35:36
4,22,26,2024-02-06,2024-02-06 20:37:44,2024-02-06 21:04:28
...,...,...,...,...,...
1159,12,21,2024-05-07,2024-05-07 17:37:34,2024-05-07 17:59:23
1160,17,21,2024-05-07,2024-05-07 17:41:37,2024-05-07 18:03:27
1161,17,18,2024-05-07,2024-05-07 18:23:10,2024-05-07 18:41:16
1162,12,20,2024-05-07,2024-05-07 18:30:04,2024-05-07 18:50:31


In [24]:
suma=0
for i in range(len(e)):
    suma=suma+(pd.Timestamp(e['updated_at'].values[i])-pd.Timestamp(e['created_at'].values[i])).seconds/60

In [25]:
suma/len(e)

34.92595933562433

In [63]:
import math 
def mformat(x):
    if(x>60):
        return(str(math.floor(x/60))+'hrs '+str(x%60)+'min')
    else:
        return(str(x)+'min')
for i in e['aplica'].unique():
    suma=0
    e_i=e.loc[e['aplica']==i]
    print('ENCUESTADOR: ',i,'media (minutos por encuesta)',e_i['tiempo'].mean())
    print('','Tiempo maximo:',mformat(e_i['tiempo'].max()))
    encuesta=e_i.loc[e_i['tiempo']==e_i['tiempo'].max()]
    print(encuesta[['aplica','cuenta','fec_capt','created_at','updated_at']])
    print('','Teimpo minimo',mformat(e_i['tiempo'].min()))
    
    encuesta=e_i.loc[e_i['tiempo']==e_i['tiempo'].min()]
    print(encuesta[['registro','aplica','cuenta','fec_capt','created_at','updated_at']])
    
    print()
    

ENCUESTADOR:  22 media (minutos por encuesta) 29.354430379746834
 Tiempo maximo: 8hrs 17min
    aplica     cuenta   fec_capt          created_at          updated_at
475     22  312355120 2024-03-13 2024-03-13 10:17:48 2024-03-13 18:35:21
 Teimpo minimo 9min
    registro aplica     cuenta   fec_capt          created_at  \
33       100     22  416030367 2024-02-12 2024-02-12 18:07:33   

            updated_at  
33 2024-02-13 18:16:48  

ENCUESTADOR:  15 media (minutos por encuesta) 16.934272300469484
 Tiempo maximo: 4hrs 29min
    aplica     cuenta   fec_capt          created_at          updated_at
565     15  416118539 2024-03-20 2024-03-20 13:02:36 2024-03-20 17:32:33
 Teimpo minimo 8min
     registro aplica     cuenta   fec_capt          created_at  \
387       478     15  306033461 2024-03-06 2024-03-06 17:26:39   
856       987     15  314250124 2024-04-16 2024-04-16 16:01:56   

             updated_at  
387 2024-03-06 17:35:30  
856 2024-04-16 16:10:49  

ENCUESTADOR:  17 media (

In [44]:
e.loc[e['tiempo']==e['tiempo'].min(),['cuenta','fec_capt','tiempo']]

,cuenta,fec_capt,tiempo
387,306033461,2024-03-06,8
856,314250124,2024-04-16,8


In [51]:
from datetime import datetime, timedelta
pd.to_datetime(e['created_at'].values[0])+timedelta(hours=-6)

Timestamp('2024-02-02 14:26:57')

In [50]:
e['created_at'].values[0]

numpy.datetime64('2024-02-02T20:26:57.000000000')

In [55]:
e['created_at']=pd.to_datetime(e['created_at']+timedelta(hours=-6))
e['updated_at']=pd.to_datetime(e['updated_at']+timedelta(hours=-6))

In [69]:
e[e['tiempo']<=10][['registro','cuenta','aplica','fec_capt','tiempo']]

,registro,cuenta,aplica,fec_capt,tiempo
33,100,416030367,22,2024-02-12,9
58,127,308160417,15,2024-02-14,9
387,478,306033461,15,2024-03-06,8
424,516,417072988,15,2024-03-08,10
505,600,415035596,15,2024-03-14,10
542,639,411044992,15,2024-03-19,10
585,683,312152613,15,2024-03-21,10
627,732,304220726,15,2024-04-02,10
711,825,417112987,12,2024-04-08,10
717,833,87381762,15,2024-04-08,10
